In [4]:
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
from tensorflow.keras.backend import expand_dims
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import joblib

In [6]:
datagen = ImageDataGenerator(
    rescale = 1./255, # need to remember we have normalised here so don't do it within neural networks
    rotation_range = 5,
    zoom_range = (0.90, 0.90),
    brightness_range = (0.95, 0.95),
    horizontal_flip = True,
    vertical_flip = True,
    data_format = 'channels_last',
    validation_split = 0.2,
    dtype = tf.bfloat16
)

In [7]:
train_generator = datagen.flow_from_directory(
    '/home/iman/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train',
    target_size = (208, 176),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'training',
    seed = 123
)

FileNotFoundError: [Errno 2] No such file or directory: '/home/iman/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train'

In [ ]:
validation_generator = datagen.flow_from_directory(
    '/home/iman/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train',
    target_size = (208, 176),
    batch_size = (32),
    class_mode = 'categorical',
    shuffle = True,
    subset = 'validation',
    seed = 123
)


In [8]:
model = Sequential()
model.add(layers.Conv2D(16, (3,3), input_shape=(208, 176, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(32, (2,2), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='AUC')
es = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[es], verbose=1)

NameError: name 'train_generator' is not defined

In [9]:
model.evaluate(validation_generator)


NameError: name 'validation_generator' is not defined

In [10]:
!pwd

/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/notebooks


In [11]:
joblib.dump(model, '../bestmodel.joblib')

TypeError: cannot pickle 'weakref' object

In [ ]:
def preprocessing(img_path):
    img = Image.open('/Users/LG/code/katarzyna-kupczyk/alzheimers_final_project/raw_data/AlzheimersDataset/train/MildDemented/mildDem54.jpg').convert('L').resize((224, 224), Image.ANTIALIAS)
    img = np.array(img)
    img_preproc = img/255
    return img_preproc

def predict(img_preproc, model_path):
    model = joblib.load(model_path)
    prediction = model.predict(img_preproc)
    return prediction